# 智能招聘面试模拟系统

本讲义结合项目代码与文档，梳理系统架构、核心技术要点及关键实现示例，适用于课程讲解与实战演练。

## 1. 系统架构与技术选型
- 前端：Vue.js + Bootstrap
- 后端：Flask RESTful API
- 数据库：SQLite
- AI服务：GLM-4-plus（大语言模型）、Whisper（语音识别）
- 定时任务：schedule库

![系统架构图](./images/interview-arch.svg)

## 2. 数据库设计与初始化

核心表结构包括岗位、候选人、面试、面试问题、面试报告等。

![数据库表设计](./images/db-er.png)

**示例：数据库初始化**

In [ ]:
import sqlite3
conn = sqlite3.connect('interview_system.db')
cursor = conn.cursor()
# 创建岗位表
cursor.execute('''
CREATE TABLE IF NOT EXISTS jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT
)
''')
# 创建候选人表
cursor.execute('''
CREATE TABLE IF NOT EXISTS candidates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    resume TEXT
)
''')
conn.commit()
conn.close()

## 3. Flask后端API核心示例

![API设计](./images/api-list.png)

**岗位管理API示例**

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)
@app.route('/api/jobs', methods=['POST'])
def create_job():
    data = request.json
    # 省略数据库插入逻辑
    return jsonify({'msg': 'Job created', 'data': data})
# app.run(port=8000)  # 启动服务（实际部署时使用）

## 4. AI面试问题与评估自动生成

集成大语言模型API，自动生成面试问题和评估报告。

**示例：调用OpenAI/GLM API生成面试问题**

In [ ]:
from openai import OpenAI

# 初始化OpenAI客户端
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

response = client.chat.completions.create(
        model="gpt-4", # 或其他适合的模型=
        messages=[
            {"role": "system", "content": "你是一名专业的招聘面试官，请根据岗位要求和候选人简历生成5个针对性的技术面试问题，每个问题附带评分标准,返回标准的json格式。"},
            {"role": "user", "content": f"岗位名称: {position_name}\n岗位要求: {requirements}\n岗位职责: {responsibilities}\n候选人简历: {resume_text}\n\n请生成10个面试问题和评分标准，JSON格式参考 {json_format} ，每个问题满分10分。"}
        ],
        response_format={"type": "json_object"},
        stream = False
    )
    
 # 解析响应内容
questions_json = response.choices[0].message.content

## 5. 定时任务与自动化处理

使用schedule库定时检查并处理待生成的面试问题和报告。

**示例：每5分钟自动执行任务**

In [ ]:
import schedule
import time
def job():
    print("自动生成面试问题和报告...")
schedule.every(5).minutes.do(job)
# while True:
#     schedule.run_pending()
#     time.sleep(1)

## 6. 语音面试与Whisper语音识别

**示例：调用Whisper模型进行语音转写**

In [ ]:
import whisper
model = whisper.load_model('base')
result = model.transcribe('audio.mp3')
print(result['text'])

## 7. 前后端联动与接口联调

前端通过HTTP请求与后端API交互，完成岗位、候选人、面试等管理。

**示例：前端fetch请求调用API**
```javascript
fetch('/api/jobs', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({ title: 'Python开发', description: '负责后端开发' })
})
  .then(res => res.json())
  .then(data => console.log(data));
```

**练习建议：**
- 尝试扩展岗位表结构，增加更多字段
- 实现面试流程的完整API
- 集成更多AI模型或优化现有流程